# `Analysis of SYNTHEA data`

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-dark')
 
from sqlalchemy import create_engine,text
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'synthea'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")
def sql(query , engine = engine):
    return pd.read_sql(query,engine)


def select(table_name):
    return sql(f'select * from {table_name} limit 5')


def run(query):
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [22]:
select('encounters').head(1)

,id,start,stop,patient,organization,provider,payer,class,description,basic_cost,total_cost,payer_coverage,reason
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23 17:45:28,2010-01-23 18:10:28,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,ambulatory,encounter for symptom,129.16,129.16,54.16,acute bronchitis (disorder)


In [23]:
readmission_encounter_id = sql(
    '''
    with duplicate as (
    select first.id,date(first.start) as prev_date, date(date_add(first.start, interval 30 day)) as after_30_days,
    second.id as next_id, date(second.start) as next_date
    from encounters first
    join encounters second
    where first.patient = second.patient and
    second.start > first.start and
    second.start <= date_add(first.start,interval 30 day) )
    select distinct next_id from duplicate;
'''
)

In [24]:
query = '''
    create table readmission_data as
    select * from encounters;
'''
run(query)

In [25]:
run('''
    alter table readmission_data
    add column readmission tinyint(1) default 0;    
''')

In [26]:
ids = "', '".join(readmission_encounter_id['next_id'])
ids = f"'{ids}'"      # wrap with quotes

query = f"""
UPDATE readmission_data
SET readmission = 1
WHERE id IN ({ids});
"""

run(query)

In [28]:
select('readmission_data').head(1)

,id,start,stop,patient,organization,provider,payer,class,description,basic_cost,total_cost,payer_coverage,reason,readmission
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23 17:45:28,2010-01-23 18:10:28,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,ambulatory,encounter for symptom,129.16,129.16,54.16,acute bronchitis (disorder),0


`MAIN DATASET IS CREATED FOR ANALYSIS`

`IMP TABLES FOR PROBLEM ->`

`MEDICATIONS`
`PATIENTS`
`CONDITIONS`
`PROCEDURES`
`CAREPLANS`

In [29]:
select('medications')

,start,stop,patient,payer,encounter,medicine_code,base_cost,payer_coverage,dispenses,total_cost,reason
0,2010-05-05 00:26:23,2011-04-30 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,677.08,0.0,12,8124.96,unknown
1,2011-04-30 00:26:23,2012-04-24 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,624.09,0.0,12,7489.08,unknown
2,2012-04-24 00:26:23,2013-04-19 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,43.32,0.0,12,519.84,unknown
3,2011-05-13 12:58:08,2011-05-27 12:58:08,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,8.14,0.0,1,8.14,acute bronchitis (disorder)
4,2011-12-08 15:02:18,2011-12-22 15:02:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,11.91,0.0,1,11.91,viral sinusitis (disorder)


In [41]:
query = '''
    select round(max(datediff(stop,start))) as medication_duration,
    encounter,
    count(distinct medicine_code) as medicins,
    sum(base_cost) as base_cost,
    sum(payer_coverage) as payer_coverage,
    sum(total_cost) as total_cost
    from medications
    group by encounter
'''

medications_agg = sql(query)

In [43]:
medications_agg.head()

,medication_duration,encounter,medicins,base_cost,payer_coverage,total_cost
0,189.0,0000d0b7-937c-498a-9da4-32a5d29dee39,2,541.11,0.00,3246.66
1,371.0,0004e2e7-e3f2-4d25-b3eb-813710f2f9bc,1,263.49,203.49,3161.88
2,63.0,00052e41-7581-46e1-8c7f-d31bbb50ae72,2,866.59,716.59,1733.18
3,0.0,00055b87-0a03-4ca8-a69d-924bbb84033a,1,263.49,0.00,263.49
4,360.0,0005b0a0-1b05-40ec-a741-11c116654840,1,399.22,0.00,4790.64


In [44]:
select('patients')

,id,birthdate,marital,race,ethnicity,gender,city,state,county,expenses,coverage
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,m,white,hispanic,m,chicopee,massachusetts,hampden county,271227.08,1334.88
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,m,white,nonhispanic,m,somerville,massachusetts,middlesex county,793946.01,3204.49
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,m,white,nonhispanic,m,chicopee,massachusetts,hampden county,574111.90,2606.40
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,m,white,nonhispanic,f,lowell,massachusetts,middlesex county,935630.30,8756.19
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,u,white,nonhispanic,m,boston,massachusetts,suffolk county,598763.07,3772.20


In [61]:
query = '''
    select id,
    (timestampdiff(year,birthdate,"2020-12-31")) as age,
    marital,race,ethnicity,gender
    from patients
'''

patient_agg = sql(query)

In [62]:
select('conditions')

,start,stop,patient,encounter,description
0,2001-05-01,None,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,chronic sinusitis (disorder)
1,2011-08-09,2011-08-16,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,9d35ec9f-352a-4629-92ef-38eae38437e7,viral sinusitis (disorder)
2,2011-11-16,2011-11-26,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd,acute viral pharyngitis (disorder)
3,2011-05-13,2011-05-27,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,acute bronchitis (disorder)
4,2011-02-06,2011-02-14,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,acute viral pharyngitis (disorder)


In [69]:
select('backup_conditions')

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2001-05-01,None,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,40055000,Chronic sinusitis (disorder)
1,2011-08-09,2011-08-16,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,9d35ec9f-352a-4629-92ef-38eae38437e7,444814009,Viral sinusitis (disorder)
2,2011-11-16,2011-11-26,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd,195662009,Acute viral pharyngitis (disorder)
3,2011-05-13,2011-05-27,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,10509002,Acute bronchitis (disorder)
4,2011-02-06,2011-02-14,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,195662009,Acute viral pharyngitis (disorder)


In [73]:
query = '''
    select encounter, count(distinct description) as total_conditions from conditions
    group by encounter
'''
conditions_agg = sql(query)

In [74]:
select('procedures')

,date,patient,encounter,description,base_cost,reason
0,2011-04-30t00:26:23z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,insertion of subcutaneous contraceptive,14896.56,unknow
1,2010-07-27t12:58:08z,10339b10-3cd1-4ac3-ac13-ec26728cb592,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,medication reconciliation (procedure),726.51,unknow
2,2010-11-20t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,7ff86631-0378-4bfc-92ce-1edd697eb18e,medication reconciliation (procedure),788.50,unknow
3,2011-02-07t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,throat culture (procedure),2070.44,acute viral pharyngitis (disorder)
4,2011-04-19t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,640837d9-845a-433c-9fad-47426664a69d,throat culture (procedure),2479.39,acute viral pharyngitis (disorder)


In [80]:
query = '''
    select encounter, 
    count(distinct description) as total_procedures,
    count(distinct reason) as total_reasons,
    sum(base_cost) as total_procedure_cost
    from procedures
    group by encounter
'''
procedures_agg = sql(query)

In [81]:
select('careplans')

,id,start,stop,patient,encounter,description,reason
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,respiratory therapy,acute bronchitis (disorder)
1,07d9ddd8-dfa1-4e43-9bfe-39f63f4ace15,2011-05-13,2011-08-02,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,respiratory therapy,acute bronchitis (disorder)
2,a3bb6e99-3b99-44b3-974c-e230b4511b5c,2011-12-31,2012-11-30,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,16300c56-a035-4126-a656-68c093da6dfc,respiratory therapy,acute bronchitis (disorder)
3,9f5284b7-425a-486a-b36e-ab818c018f2f,2016-12-29,2017-01-05,034e9e3b-2def-4559-bb2a-7850888ae060,3b639086-5fbc-4720-8c31-e8c8c0f1d660,respiratory therapy,acute bronchitis (disorder)
4,47ede16c-c216-4f81-a16b-0e858de9cdc3,2017-01-22,2017-02-12,10339b10-3cd1-4ac3-ac13-ec26728cb592,4ec8d55b-05fc-42a5-bfa3-1e233874a362,wound care,laceration of foot


In [93]:
query = '''
        select 
        encounter,
        round(max(datediff(stop,start))) as careplan_duration,
        count(distinct description) as total_descriptions,
        count(distinct reason) as total_reasons,
        count(distinct id) as total_palns
        from careplans 
        group by encounter 
'''

careplans_agg = sql(query)

In [94]:
careplans_agg

,encounter,careplan_duration,total_descriptions,total_reasons,total_palns
0,000fef3b-ba44-4b01-952e-7eeb8a5732a8,67.0,1,1,1
1,001d66d9-b1bc-425b-a3a7-3c24de8fcb16,217.0,1,1,1
2,001ed2aa-e772-4407-9902-38ef70bd1a1e,30.0,1,1,1
3,005ba3e5-d0f6-4bed-a1e7-a309cac30be2,NaN,1,1,1
4,006f8752-ea4b-4cd3-bda5-9ae9aede145d,7.0,1,1,1
...,...,...,...,...,...
3467,ffbe0c16-ef04-4981-bf7f-c22ac759f3db,28.0,1,1,1
3468,ffc04e78-b296-4443-a3a8-8c92e4b5c6e8,14.0,1,1,1
3469,ffc24e2b-6c46-4d95-8b5d-584e03595d2d,7.0,1,1,1
3470,ffd729e5-e8cc-451b-8fde-791e08fef533,311.0,1,1,1
